# AutoGen RoundRobinGroupChat
A team is a group of agents that work together to achieve a common goal.


## Azure Resources Needed
1. Azure OpenAI
    - Deploy GPT-4o


## Load Azure Configurations

In [2]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client
Using the model client class

In [3]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)

## Creating a Team

RoundRobinGroupChat is a team configuration where all agents share the same context and take turns responding in a round-robin fashion. Each agent, during its turn, broadcasts its response to all other agents, ensuring that the entire team maintains a consistent context.

We create a team with two AssistantAgent and a TextMentionTermination condition that stops the team when a specific word is detected in the agent’s response.

In [4]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken


# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=az_model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=az_model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

## Running a Team
Call the run() method to start the team with a task.

The termination condition was met when the word “APPROVE” is detected in the agent’s response. When the team stops, it returns a TaskResult object with all the messages produced by the agents in the team.

In [ ]:
# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
result = await team.run(task="Write a short poem about the fall season.")
print(result)

## Observing a Team
Similar to the agent’s on_messages_stream() method, you can stream the team’s messages while it is running by calling the run_stream() method. This method returns a generator that yields messages produced by the agents in the team as they are generated, with the final item being the TaskResult object.

In [ ]:
# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

## Console()
The Console() method provides a convenient way to print messages to the console with proper formatting.

In [ ]:
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a short poem about the fall season."))  # Stream the messages to the console.

## Team has a State

In [ ]:
await Console(team.run_stream(task="What was the last message?"))  # Stream the messages to the console.

## Resetting a Team
You can reset the team by calling the reset() method. This method will clear the team’s state, including all agents. It will call the each agent’s on_reset() method

In [9]:
await team.reset()  # Reset the team for the next run.

## Stopping a Team
You can also stop the team from outside by using the ExternalTermination.

Calling set() on ExternalTermination will stop the team when the current agent’s turn is over. 

Thus, the team may not stop immediately. This allows the current agent to finish its turn and broadcast the final message to the team before the team stops, keeping the team’s state consistent.

In [ ]:
# Create a new team with an external termination condition.
external_termination = ExternalTermination()
team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition=external_termination | text_termination,  # Use the bitwise OR operator to combine conditions.
)

# Run the team in a background task.
run = asyncio.create_task(Console(team.run_stream(task="Write a short poem about the fall season.")))

# Wait for some time.
await asyncio.sleep(0.1)

# Stop the team.
external_termination.set()

# Wait for the team to finish.
await run # Primary agent is still allowed to complete its task.

## Resuming a Team
Teams are stateful and maintains the conversation history and context after each run, unless you reset the team.

You can resume a team to continue from where it left off by calling the run() or run_stream() method again without a new task. RoundRobinGroupChat will continue from the next agent in the round-robin order.

In this example, critic agent continued where primary agent left off

In [ ]:
await Console(team.run_stream())  # Resume the team to continue the last task.

## Aborting a Team
You can abort a call to run() or run_stream() during execution by setting a CancellationToken passed to the cancellation_token parameter.

Different from stopping a team, aborting a team will immediately stop the team and raise a CancelledError exception.

In [ ]:
# Create a cancellation token.
cancellation_token = CancellationToken()

# Use another coroutine to run the team.
run = asyncio.create_task(
    team.run(
        task="Translate the poem to Spanish.",
        cancellation_token=cancellation_token,
    )
)

# Cancel the run.
cancellation_token.cancel()

try:
    result = await run  # This will raise a CancelledError.
except asyncio.CancelledError:
    print("Task was cancelled.")

## Other Built-In Termination Conditions:
- MaxMessageTermination: Stops after a specified number of messages have been produced, including both agent and task messages.
- TextMentionTermination: Stops when specific text or string is mentioned in a message (e.g., “TERMINATE”).
- ExternalTermination: Enables programmatic control of termination from outside the run. This is useful for UI integration (e.g., “Stop” buttons in chat interfaces).
- TokenUsageTermination: Stops when a certain number of prompt or completion tokens are used. This requires the agents to report token usage in their messages.
- TimeoutTermination: Stops after a specified duration in seconds.
- SourceMatchTermination: Stops after a specific agent responds.
- HandoffTermination: Stops when a handoff to a specific target is requested. Handoff messages can be used to build patterns such as Swarm. This is useful when you want to pause the run and allow application or user to provide input when an agent hands off to them.
- StopMessageTermination: Stops when a StopMessage is produced by an agent.

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination, ExternalTermination, TokenUsageTermination, TimeoutTermination, SourceMatchTermination, HandoffTermination, StopMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

# Create the first agent.
first_agent = AssistantAgent(
    "first",
    model_client=az_model_client,
    system_message="You will give a random number between 1 and 10.",
)

# Create the second agent.
second_agent = AssistantAgent(
    "second",
    model_client=az_model_client,
    system_message="You will give a random number between 1 and 10.",
)

# Create the second agent.
third_agent = AssistantAgent(
    "third",
    model_client=az_model_client,
    system_message="You will give a random number between 1 and 10.",
)

# MaxMessageTermination
termination = MaxMessageTermination(5) # terminate after 5 messages
team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)

await Console(team.run_stream(task="Give a number"))

In [ ]:
#TextMentionTermination
termination = TextMentionTermination("4") # Terminate when the message contains "4".
team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)
await Console(team.run_stream(task="Give a number"))

In [ ]:
import asyncio
#ExternalTermination
termination = ExternalTermination() # Create an external termination condition.

team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)

# Run the team in a background task.
run = asyncio.create_task(Console(team.run_stream(task="Give a number")))

# Wait for some time.
await asyncio.sleep(3)

# Stop the team.
termination.set()

# Wait for the team to finish.
await run


In [ ]:
#TokenUsageTermination
termination = TokenUsageTermination(1000) # terminate when token usage is met.
team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)
await Console(team.run_stream(task="Give a number"))

In [ ]:
#TimeoutTermination
termination = TimeoutTermination(5) # terminate when time out is met.
team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)
await Console(team.run_stream(task="Give a number"))

In [ ]:
#SourceMatchTermination
termination = SourceMatchTermination("third") # terminate when source match agent is met.
team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)
await Console(team.run_stream(task="Give a number"))